In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
import os

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/mini_proj_data/processed_data.csv')
df

In [ ]:
df.columns

# 1. With Processed Images 

In [ ]:
X = df['clean_path']   # paths to images

In [ ]:
import cv2
import numpy as np

def load_and_preprocess_image(path, target_size=(224,224)):
    img = cv2.imread(path)                      # read image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
    img = cv2.resize(img, target_size)
    img = img / 255.0                           # normalize to [0,1]
    return img.astype('float32')


X_images=np.array([load_and_preprocess_image(p) for p in X])

# --- Standardization step (per-channel) ---
mean = np.mean(X_images, axis=(0,1,2), keepdims=True)
std  = np.std(X_images, axis=(0,1,2), keepdims=True)

X_images = (X_images - mean) / (std + 1e-7)


In [ ]:
X_images

In [ ]:
from tensorflow.keras import Model, regularizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dropout, Input
import tensorflow as tf

# ---------------- CNN1 ----------------
def build_cnn1(input_shape):
    inputs = Input(shape=input_shape)

    x = Conv2D(32, (3,3), activation='relu', padding="same",
               kernel_regularizer=regularizers.l2(1e-4))(inputs)
    x = MaxPooling2D((2,2))(x)

    x = Conv2D(64, (3,3), activation='relu', padding="same",
               kernel_regularizer=regularizers.l2(1e-4))(x)
    x = MaxPooling2D((2,2))(x)

    x = Conv2D(128, (3,3), activation='relu', padding="same",
               kernel_regularizer=regularizers.l2(1e-4))(x)
    x = MaxPooling2D((2,2))(x)

    x = Conv2D(256, (3,3), activation='relu', padding="same",
               kernel_regularizer=regularizers.l2(1e-4))(x)
    x = MaxPooling2D((2,2))(x)
    
    x = Flatten()(x)
    x = Dropout(0.5)(x)   # still useful to avoid overfitting
    return Model(inputs, x, name="CNN1")

# ---------------- CNN2 ----------------
def build_cnn2(input_shape):
    inputs = Input(shape=input_shape)

    x = Conv2D(256, (7,7), activation='relu', padding="same",
               kernel_regularizer=regularizers.l2(1e-4))(inputs)
    x = AveragePooling2D((2,2))(x)

    x = Conv2D(128, (5,5), activation='relu', padding="same",
               kernel_regularizer=regularizers.l2(1e-4))(x)
    x = AveragePooling2D((2,2))(x)

    x = Conv2D(96, (3,3), activation='relu', padding="same",
               kernel_regularizer=regularizers.l2(1e-4))(x)
    x = AveragePooling2D((2,2))(x)

    x = Conv2D(96, (3,3), activation='relu', padding="same",
               kernel_regularizer=regularizers.l2(1e-4))(x)
    x = AveragePooling2D((2,2))(x)

    x = Flatten()(x)
    x = Dropout(0.5)(x)
    return Model(inputs, x, name="CNN2")

# ---------------- CNN3 ----------------
def build_cnn3(input_shape):
    inputs = Input(shape=input_shape)

    x = Conv2D(32, (3,3), activation='relu', padding="same",
               kernel_regularizer=regularizers.l2(1e-4))(inputs)
    x = MaxPooling2D((2,2))(x)

    x = Conv2D(96, (5,5), activation='relu', padding="same",
               kernel_regularizer=regularizers.l2(1e-4))(x)
    x = MaxPooling2D((2,2))(x)

    x = Conv2D(128, (5, 5), activation='relu', padding="same",
               kernel_regularizer=regularizers.l2(1e-4))(x)
    x = MaxPooling2D((2,2))(x)

    x = Conv2D(256, (7, 7), activation='relu', padding="same",
               kernel_regularizer=regularizers.l2(1e-4))(x)
    x = MaxPooling2D((2,2))(x)
    
    x = Flatten()(x)
    x = Dropout(0.5)(x)   # still useful to avoid overfitting
    return Model(inputs, x, name="CNN1")


# ---------------- CNN4 ----------------
def build_cnn4(input_shape):
    inputs = Input(shape=input_shape)

    x = Conv2D(32, (3,3), activation='relu', padding="same",
               kernel_regularizer=regularizers.l2(1e-4))(inputs)
    x = AveragePooling2D((3,3))(x)

    x = Conv2D(32, (3,3), activation='relu', padding="same",
               kernel_regularizer=regularizers.l2(1e-4))(x)
    x = AveragePooling2D((5,5))(x)

    x = Conv2D(64, (5,5), activation='relu', padding="same",
               kernel_regularizer=regularizers.l2(1e-4))(x)
    x = AveragePooling2D((3,3))(x)

    x = Conv2D(128, (5,5), activation='relu', padding="same",
               kernel_regularizer=regularizers.l2(1e-4))(x)
    x = AveragePooling2D((3,3))(x)

    x = Flatten()(x)
    x = Dropout(0.5)(x)
    return Model(inputs, x, name="CNN2")


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
# datagen.fit(X_train_images)


In [ ]:
input_shape = (224, 224, 3)
CNN1 = build_cnn1(input_shape)
CNN2 = build_cnn2(input_shape)
CNN3 = build_cnn3(input_shape)
CNN4 = build_cnn4(input_shape)
print("---- CNN1 Summary ----")
CNN1.summary()

print("\n---- CNN2 Summary ----")
CNN2.summary()

print("\n---- CNN3 Summary ----")
CNN3.summary()

print("\n---- CNN4 Summary ----")
CNN4.summary()
# Use augmented data for feature extraction
# train_generator = datagen.flow(X_train_images, y_train, batch_size=32, shuffle=False)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [ ]:
features_cnn1 = CNN1.predict(X_images, verbose=1)
features_cnn2 = CNN2.predict(X_images, verbose=1)
features_cnn3 = CNN3.predict(X_images, verbose=1)
features_cnn4 = CNN4.predict(X_images, verbose=1)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
all_features = np.concatenate([features_cnn1, features_cnn2], axis=1)
features_pca = pca.fit_transform(all_features)

# later you can split back if needed


In [ ]:
from sklearn.decomposition import FactorAnalysis

# Concatenate features from CNN3 and CNN4
all_features_dual2 = np.concatenate([features_cnn3, features_cnn4], axis=1)

# Apply Factor Analysis to reduce dimensionality
fa = FactorAnalysis(n_components=50, random_state=42)
features_fa = fa.fit_transform(all_features_dual2)

# features_fa now contains the reduced features for dual2

In [ ]:
# Step 1: Merge features from both Dual CNNs
# features_pca: reduced features from Dual-1 (PCA)
# features_fa: reduced features from Dual-2 (FA)
merged_features = np.concatenate([features_pca, features_fa], axis=1)

# Step 2: Remove duplicate features (columns)
# Convert to DataFrame for easy duplicate removal
import pandas as pd
merged_df = pd.DataFrame(merged_features)
# Remove duplicate columns
merged_df = merged_df.loc[:, ~merged_df.T.duplicated()]




# Step 1: Train + temp (val+test)
X_train, X_temp, y_train, y_temp = train_test_split(
    merged_df.values, df['label'].values, test_size=0.3, random_state=42, stratify=y
)

# Step 2: Validation + Test (split from temp)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=1/3, random_state=42, stratify=y_temp
)

print("Train:", X_train.shape, y_train.shape)
print("Val:  ", X_val.shape, y_val.shape)
print("Test: ", X_test.shape, y_test.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
# ---------------- Classifier ----------------
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(7, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()
# Early stopping callback



In [ ]:
history = model.fit(
    X_train, y_train,   # <-- use your numpy labels directly
    validation_data=(X_val, y_val),
    epochs=80,
    batch_size=128,
    callbacks=[early_stop], 
       # <- stops when val_acc stops improving
    verbose=1
)

In [ ]:
from sklearn.metrics import classification_report  
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.preprocessing import label_binarize
def metrics():
    # Evaluate model on test data
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
    print(f"✅ Test Accuracy: {test_acc:.4f}")
    print(f"✅ Test Loss: {test_loss:.4f}")
    
    y_pred_probs = model.predict(X_test)             # probabilities (N, 7)
    y_pred = np.argmax(y_pred_probs, axis=1)  
    

    print("\n📊 Classification Report:")
    print(classification_report(y_test, y_pred, digits=4)) 
    cm = confusion_matrix(y_test, y_pred)

    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=[f"Class {i}" for i in range(7)],
                yticklabels=[f"Class {i}" for i in range(7)])
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.show()
    # Binarize test labels (one-hot for ROC)
    y_test_bin = label_binarize(y_test, classes=np.arange(7))

    # ROC-AUC (macro average)
    roc_auc = roc_auc_score(y_test_bin, y_pred_probs, multi_class="ovr")
    print(f" ROC-AUC (macro): {roc_auc:.4f}")
    


In [ ]:
def save_model(model, model_name="dual_cnn_classifier"):
    model_dir = "/content/drive/MyDrive/mini_proj_data/saved_models/"
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_path = os.path.join(model_dir, model_name + ".h5")
    model.save(model_path)
    print(f"Model saved to {model_path}")



# 2. Processed images with data splitting outside

In [ ]:
from sklearn.model_selection import train_test_split

# Step 1: Split paths and labels first
paths = df['clean_path'].values
labels = df['label'].values

# 70% train, 30% temp
paths_train, paths_temp, labels_train, labels_temp = train_test_split(
    paths, labels, test_size=0.3, random_state=42, stratify=labels
)
# 20% test, 10% val from temp
paths_test, paths_val, labels_test, labels_val = train_test_split(
    paths_temp, labels_temp, test_size=1/3, random_state=42, stratify=labels_temp
)

# Step 2: Preprocess images for each split
X_train_images = np.array([load_and_preprocess_image(p) for p in paths_train])
X_test_images = np.array([load_and_preprocess_image(p) for p in paths_test])
X_val_images = np.array([load_and_preprocess_image(p) for p in paths_val])

# Step 3: Feature extraction for each split
features_cnn1_train = CNN1.predict(X_train_images, verbose=1)
features_cnn2_train = CNN2.predict(X_train_images, verbose=1)
# ... repeat for test and val sets

# Step 4: Dimensionality reduction, merging, deduplication for each split
# (Apply PCA/FA on train, transform test/val using fitted models)

# Step 5: Train classifier on train set, evaluate on test and val sets
# (Use metrics like accuracy, confusion matrix, etc.)

In [ ]:
# Step 4: Dimensionality reduction, merging, deduplication for each split

from sklearn.decomposition import PCA, FactorAnalysis

# Fit PCA and FA on train set only
pca = PCA(n_components=50, random_state=42)
fa = FactorAnalysis(n_components=50, random_state=42)

# Extract features for all CNNs and splits
features_cnn1_train = CNN1.predict(X_train_images, verbose=1)
features_cnn2_train = CNN2.predict(X_train_images, verbose=1)
features_cnn3_train = CNN3.predict(X_train_images, verbose=1)
features_cnn4_train = CNN4.predict(X_train_images, verbose=1)

features_cnn1_test = CNN1.predict(X_test_images, verbose=1)
features_cnn2_test = CNN2.predict(X_test_images, verbose=1)
features_cnn3_test = CNN3.predict(X_test_images, verbose=1)
features_cnn4_test = CNN4.predict(X_test_images, verbose=1)

features_cnn1_val = CNN1.predict(X_val_images, verbose=1)
features_cnn2_val = CNN2.predict(X_val_images, verbose=1)
features_cnn3_val = CNN3.predict(X_val_images, verbose=1)
features_cnn4_val = CNN4.predict(X_val_images, verbose=1)

# Dual-1: PCA on CNN1+CNN2
all_features_train_dual1 = np.concatenate([features_cnn1_train, features_cnn2_train], axis=1)
features_pca_train = pca.fit_transform(all_features_train_dual1)

all_features_test_dual1 = np.concatenate([features_cnn1_test, features_cnn2_test], axis=1)
features_pca_test = pca.transform(all_features_test_dual1)

all_features_val_dual1 = np.concatenate([features_cnn1_val, features_cnn2_val], axis=1)
features_pca_val = pca.transform(all_features_val_dual1)

# Dual-2: FA on CNN3+CNN4
all_features_train_dual2 = np.concatenate([features_cnn3_train, features_cnn4_train], axis=1)
features_fa_train = fa.fit_transform(all_features_train_dual2)

all_features_test_dual2 = np.concatenate([features_cnn3_test, features_cnn4_test], axis=1)
features_fa_test = fa.transform(all_features_test_dual2)

all_features_val_dual2 = np.concatenate([features_cnn3_val, features_cnn4_val], axis=1)
features_fa_val = fa.transform(all_features_val_dual2)

# Merge features
merged_train = np.concatenate([features_pca_train, features_fa_train], axis=1)
merged_test = np.concatenate([features_pca_test, features_fa_test], axis=1)
merged_val = np.concatenate([features_pca_val, features_fa_val], axis=1)

# Remove duplicate columns
import pandas as pd
merged_train_df = pd.DataFrame(merged_train)
merged_test_df = pd.DataFrame(merged_test)
merged_val_df = pd.DataFrame(merged_val)

# Use train columns to remove duplicates
unique_cols = ~merged_train_df.T.duplicated()
merged_train_df = merged_train_df.loc[:, unique_cols]
merged_test_df = merged_test_df.loc[:, unique_cols]
merged_val_df = merged_val_df.loc[:, unique_cols]

In [ ]:
# Step 5: Train classifier and evaluate metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential([
    Dense(256, activation='relu', input_shape=(merged_train_df.shape[1],)),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(7, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    merged_train_df.values, labels_train,
    validation_data=(merged_val_df.values, labels_val),
    epochs=80,
    batch_size=128,
    callbacks=[early_stop],
    verbose=1
)

# Evaluate on test set
test_loss, test_acc = model.evaluate(merged_test_df.values, labels_test, verbose=1)
print(f"✅ Test Accuracy: {test_acc:.4f}")
print(f"✅ Test Loss: {test_loss:.4f}")

# Classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

y_pred_probs = model.predict(merged_test_df.values)
y_pred = np.argmax(y_pred_probs, axis=1)

print("\n📊 Classification Report:")
print(classification_report(labels_test, y_pred, digits=4))
cm = confusion_matrix(labels_test, y_pred)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=[f"Class {i}" for i in range(7)],
            yticklabels=[f"Class {i}" for i in range(7)])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

# 2. Un processed images

# 3. Unprocessed images with train-test outside arch